# Working on creating better reporting for the deferred revenue waterfall

To be done
 - Add totals to each waterfall as the first non-index column
 - Add ADobe totals to the waterfall
 - Since my reporting is by period, create a summary page that has the BUs by quarter
 - Create a quarterly summary for each BU to check input data matches
 




In [ ]:
#IMPORT STATEMENTS
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import json
import pickle

In [ ]:
# LOAD CONFIG FILES WITH FILENAMES AND PATHS
with open('../src/base_config.json') as json_file:
    config_dict1 = json.load(json_file)

with open('../src/period_config.json') as json_file:
    config_dict2 = json.load(json_file)

# Merging the two config dictionaries
config_dict = {**config_dict1, **config_dict2}


## Loading up the last saved version of all data

In [3]:
dict_final = pickle.load(open(config_dict['output_dir']['final']+'final_forecast3.p', 'rb'))

In [4]:
dict_final.keys()

dict_keys(['model_dict', 'ADBE_cal', 'bookings', 'FX_forwards', 'FX_rates', 'final', 'billings', 'forecast', 'initial_waterfall', 'bill_waterfall', 'waterfall', 'as_performed'])

In [ ]:
df_billings = dict_final['billings']
df_billings.columns

In [ ]:
wf_init = dict_final['initial_waterfall']

In [ ]:
wf_init = wf_init.rename(columns={'Total': 'Balance'})

In [ ]:
wf_init = wf_init.sort_index(axis=1)

In [ ]:
def add_Adobe_waterfall_totals(df_wf):
    df_totals = df_wf.sum(level=1, axis=0)
    df_totals = pd.concat([df_totals], keys=['Adobe'], names=['BU'])
    df_wf = pd.concat([df_wf, df_toals])
    
    df_wf = df_wf.sum(level=1, axis=0)
    this_waterfall = pd.concat([this_waterfall], keys=['Adobe'], names=['BU'])
    
    df_wf = pd.concat([df_wf, this_waterfall])
    
    return df_wf

In [ ]:
wf_init

### Now add Adobe Totals

In [ ]:
a = wf_init.index.levels[0]
a

In [ ]:
for item in a:
    print(item)
    this_df = wf_init.loc[item]
    print(this_df)

In [ ]:
this_waterfall = wf_init.sum(level=1, axis=0)
this_waterfall

In [ ]:
this_waterfall = pd.concat([this_waterfall], keys=['Adobe'], names=['BU'])
this_waterfall

In [ ]:
wf_init = pd.concat([wf_init, this_waterfall])
wf_init

## Check the DME bookings totals
These may have been overstated!!
The MPC in column 1 of the DX bookings contains Enterprise as a superset and all other fields are a subset of the 'Enterprise'. I did not understand this and double counted the market areas by including both enterprise and subsets of enterprise

In [ ]:
dict_final.keys()

In [5]:
df_bookings = dict_final['bookings']
df_bookings.head()

,BU,geo,region,country,Quarter,value,Currency
0,Creative,AMER,Latin America,Brazil,Q1_2021,1.151616e+06,USD
1,Creative,AMER,Latin America,Mexico,Q1_2021,-2.580651e+04,USD
2,Creative,AMER,Latin America,Strat. Latin America,Q1_2021,-3.929125e+04,USD
3,Creative,AMER,North America,Canada,Q1_2021,8.008077e+05,USD
4,Creative,AMER,North America,United States,Q1_2021,1.798236e+07,USD


In [6]:
df_bookings['BU'].value_counts()

Document Cloud      96
Creative            96
Experience Cloud    92
Name: BU, dtype: int64

In [7]:
df_DME = df_bookings[df_bookings['BU']=='Experience Cloud'].copy()

In [8]:
df_DME

,BU,geo,region,country,Quarter,value,Currency
48,Experience Cloud,AMER,Latin America,Brazil,Q1_2021,1.713525e+06,USD
49,Experience Cloud,AMER,North America,Canada,Q1_2021,3.962295e+06,USD
50,Experience Cloud,AMER,North America,United States,Q1_2021,9.373140e+07,USD
51,Experience Cloud,ASIA,ANZ,Aus and New Zealand,Q1_2021,5.275588e+06,AUD
52,Experience Cloud,ASIA,Greater China,China,Q1_2021,6.054372e+05,USD
...,...,...,...,...,...,...,...
279,Experience Cloud,EMEA,Southwest Europe,Benelux,Q4_2021,5.054056e+06,EUR
280,Experience Cloud,EMEA,Southwest Europe,France,Q4_2021,7.409964e+06,EUR
281,Experience Cloud,EMEA,Southwest Europe,Iberica,Q4_2021,3.049557e+06,EUR
282,Experience Cloud,EMEA,Southwest Europe,Italy,Q4_2021,2.973344e+06,EUR


In [9]:
df_DME_q3 = df_DME[df_DME['Quarter']=='Q3_2021'].copy()
df_DME_q3.head()

,BU,geo,region,country,Quarter,value,Currency
190,Experience Cloud,AMER,Latin America,Brazil,Q3_2021,2.539522e+06,USD
191,Experience Cloud,AMER,North America,Canada,Q3_2021,5.344730e+06,USD
192,Experience Cloud,AMER,North America,United States,Q3_2021,1.261139e+08,USD
193,Experience Cloud,ASIA,ANZ,Aus and New Zealand,Q3_2021,9.420692e+06,AUD
194,Experience Cloud,ASIA,Greater China,China,Q3_2021,9.838355e+05,USD


In [10]:
df_DME_q3['value'].sum()

199019291.2446458

In [11]:
df_DME_q4 = df_DME[df_DME['Quarter']=='Q4_2021'].copy()
df_DME_q4['value'].sum()

284278630.31336653

## There is an error in the bookings. bookings is fine, but the forecast may contain double the bookings

In [12]:
dict_final.keys()

dict_keys(['model_dict', 'ADBE_cal', 'bookings', 'FX_forwards', 'FX_rates', 'final', 'billings', 'forecast', 'initial_waterfall', 'bill_waterfall', 'waterfall', 'as_performed'])

In [13]:
df_fcst = dict_final['forecast']
df_fcst.columns

Index(['BU', 'curr', 'period', 'recognized_DC', 'service_DC', 'deferred_1M_DC',
       'deferred_3M_DC', 'deferred_6M_DC', 'deferred_1Y_DC', 'deferred_2Y_DC',
       'deferred_3Y_DC', 'Period_Weeks', 'r_squared', 'intercept', 'coeff',
       'X_length', 'recognized_US', 'service_US', 'deferred_1M_US',
       'deferred_3M_US', 'deferred_6M_US', 'deferred_1Y_US', 'deferred_2Y_US',
       'deferred_3Y_US', 'book_1Y_DC', 'book_1Y_US', 'is_forecast'],
      dtype='object')

In [14]:
df_fcst.head(10)

,BU,curr,period,recognized_DC,service_DC,deferred_1M_DC,deferred_3M_DC,deferred_6M_DC,deferred_1Y_DC,deferred_2Y_DC,...,service_US,deferred_1M_US,deferred_3M_US,deferred_6M_US,deferred_1Y_US,deferred_2Y_US,deferred_3Y_US,book_1Y_DC,book_1Y_US,is_forecast
0,Creative,ARS,2021-07,313318.00,0.0,6.320936e+230,0.0,0.0,2366946.53,0.0,...,0.0,6.484727e+228,0.0,0.0,24282.800269,0.0,0.0,0.0,0.0,1
1,Creative,ARS,2021-08,243158.23,0.0,6.849460e+230,0.0,0.0,3287724.88,0.0,...,0.0,6.853970e+228,0.0,0.0,32898.896274,0.0,0.0,0.0,0.0,1
2,Creative,ARS,2021-09,200164.71,0.0,9.222480e+230,0.0,0.0,5668100.40,0.0,...,0.0,9.006838e+228,0.0,0.0,55355.678349,0.0,0.0,0.0,0.0,1
3,Creative,ARS,2021-10,279809.23,0.0,7.906508e+230,0.0,0.0,2854264.08,0.0,...,0.0,7.453518e+228,0.0,0.0,26907.338343,0.0,0.0,0.0,0.0,1
4,Creative,ARS,2021-11,409672.61,0.0,8.435032e+230,0.0,0.0,3920058.84,0.0,...,0.0,7.684918e+228,0.0,0.0,35714.539790,0.0,0.0,0.0,0.0,1
5,Creative,ARS,2021-12,264486.70,0.0,1.120445e+231,0.0,0.0,6493817.52,0.0,...,0.0,9.876614e+228,0.0,0.0,57242.393353,0.0,0.0,0.0,0.0,1
6,Creative,ARS,2022-01,452047.47,0.0,9.492081e+230,0.0,0.0,4571898.58,0.0,...,0.0,8.057050e+228,0.0,0.0,38807.104212,0.0,0.0,0.0,0.0,1
7,Creative,ARS,2022-02,286928.38,0.0,1.002060e+231,0.0,0.0,3762826.75,0.0,...,0.0,8.201676e+228,0.0,0.0,30798.025196,0.0,0.0,0.0,0.0,1
8,Creative,ARS,2022-03,595940.33,0.0,1.318641e+231,0.0,0.0,5183129.66,0.0,...,0.0,1.042040e+229,0.0,0.0,40959.045614,0.0,0.0,0.0,0.0,1
9,Creative,ARS,2022-04,278621.22,0.0,1.107765e+231,0.0,0.0,5592479.30,0.0,...,0.0,8.753979e+228,0.0,0.0,44193.880873,0.0,0.0,0.0,0.0,1


In [21]:
period_list = ['2021-07', '2021-08', '2021-09']
df_q3_book = df_fcst[(df_fcst['BU']=='Experience Cloud') & (df_fcst['period'].isin(period_list))]
df_q3_book

,BU,curr,period,recognized_DC,service_DC,deferred_1M_DC,deferred_3M_DC,deferred_6M_DC,deferred_1Y_DC,deferred_2Y_DC,...,service_US,deferred_1M_US,deferred_3M_US,deferred_6M_US,deferred_1Y_US,deferred_2Y_US,deferred_3Y_US,book_1Y_DC,book_1Y_US,is_forecast
36,Experience Cloud,ARS,2021-07,0.000000e+00,0.00,0.000000e+00,0.000000e+00,0.00,0.000000e+00,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,1
37,Experience Cloud,ARS,2021-08,0.000000e+00,0.00,0.000000e+00,0.000000e+00,0.00,0.000000e+00,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,1
38,Experience Cloud,ARS,2021-09,0.000000e+00,0.00,0.000000e+00,0.000000e+00,0.00,0.000000e+00,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,1
96,Experience Cloud,AUD,2021-07,1.782501e+06,614720.00,-9.579738e+03,1.784782e+06,806319.50,4.386164e+06,0.0,...,4.757438e+05,-7.413946e+03,1.381278e+06,6.240264e+05,3.394538e+06,0.0,0.0,3.362503e+06,2.607764e+06,1
97,Experience Cloud,AUD,2021-08,2.114707e+06,344620.00,-1.489799e+04,3.787193e+06,33750.00,3.880818e+06,0.0,...,2.667493e+05,-1.153163e+04,2.931435e+06,2.612382e+04,3.003904e+06,0.0,0.0,2.975097e+06,2.307314e+06,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017,Experience Cloud,TWD,2021-08,0.000000e+00,0.00,0.000000e+00,0.000000e+00,0.00,0.000000e+00,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,1
2018,Experience Cloud,TWD,2021-09,0.000000e+00,0.00,0.000000e+00,0.000000e+00,0.00,0.000000e+00,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,1
2076,Experience Cloud,USD,2021-07,1.090305e+07,3342376.04,2.690077e+06,3.934671e+07,10871454.36,6.702188e+07,0.0,...,3.342376e+06,2.690077e+06,3.934671e+07,1.087145e+07,6.702188e+07,0.0,0.0,4.890252e+07,4.890252e+07,1
2077,Experience Cloud,USD,2021-08,9.147675e+06,2192803.99,2.646690e+06,7.944670e+06,352578.86,6.044043e+07,0.0,...,2.192804e+06,2.646690e+06,7.944670e+06,3.525789e+05,6.044043e+07,0.0,0.0,4.410036e+07,4.410036e+07,1


In [22]:
sum_q3_book = df_q3_book['book_1Y_US'].sum()
sum_q3_book

199019291.24464583

In [24]:
sum_1yr_US = df_q3_book['deferred_1Y_US'].sum()
sum_1yr_US

275665370.8244723